# Business Problem


For this project, you have been hired to produce a MySQL database on Movies from a subset of IMDB's publicly available dataset. Ultimately, you will use this database to analyze what makes a movie successful, and will provide recommendations to the stakeholder on how to make a successful movie.

## Your Stakeholder Wants More Data!
After investigating the preview of your data from Part 1, your stakeholder realized that there is no financial information included in the IMDB data (e.g. budget or revenue).

This will be a major roadblock when attempting to analyze which movies are successful and must be addressed before you will be able to determine which movies are successful.
Your stakeholder identified The Movie Database (TMDB) as a great source of financial data 

(https://www.themoviedb.org/). Thankfully, TMDB offers a free API for programmatic access to their data!


Your stakeholder wants you to extract the budget, revenue, and MPAA Rating (G/PG/PG-13/R), which is also called "Certification".

In [34]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
import os, json, math, time

from tqdm.notebook import tqdm_notebook

In [35]:
# Load API Credentials

with open('/Users/vinosraj/.secret/tmdb_api.json') as f: #default 'r' default so dont need to add
    login = json.load(f)
login.keys()

dict_keys(['api-key'])

In [36]:
# Instantiate tmdbAPI Variable
import tmdbsimple as tmdb
tmdb.API_KEY =  login['api-key']

https://developers.themoviedb.org/3/movies/get-movie-details

for API details

In [37]:
## Specify fodler for saving data
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)

In [40]:
def get_movie_with_rating(movie_id):
    #get move and release date
    movie=tmdb.Movies(movie_id)
    # construct output dict
    movie_info = movie.info()
    release = movie.releases()
    for c in release ['countries']:
        if c['iso_3166_1'] =='US': #else function &*
            movie_info['certification'] = c['certification']
        else:
            movie_info['certification'] = 'Not Available'
    return movie_info

In [41]:
def write_json(new_data, filename): 
    """Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""    
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [42]:
basics_file = "Data/title_basics.csv.gz"

basics = pd.read_csv(basics_file, low_memory=False)

basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0093119,movie,Grizzly II: Revenge,Grizzly II: The Predator,0,2020,NaN,74,"Horror,Music,Thriller"


In [43]:
# set our API call parameters 

YEARS_TO_GET = [2000,2001]


In [46]:
# Start of OUTER loop
for YEAR in tqdm_notebook (YEARS_TO_GET,desc='YEARS',position=0):
    #Defining the JSON file to store results for year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'

    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)

    ## If it does not exist: 
    if file_exists == False:

        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)

         ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)


        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to file.")


        ## save the first page of results
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)  
    ## If it exists, inform user
    else:
        print(f"[i] {JSON_FILE} already exists.")

    #Saving new year as the current df
    df = basics.loc[ basics['startYear']==YEAR].copy()
    # saving movie ids to list
    movie_ids = df['tconst'].copy()#.to_list()

    # Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)
    previous_df

    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

#Get index and movie id from list
# INNER Loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                      desc=f'Movies from {YEAR}',
                                      position=1,
                                      leave=True):
            # Attempt to retrieve then data for the movie id
        try:
            temp = get_movie_with_rating(movie_id)  #This uses your pre-made function
           # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)

            # If it fails,  make a dict with just the id and None for certification.
        except Exception as e:
                continue

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

[i] Data/tmdb_api_results_2000.json not found. Saving empty list to file.


Movies from 2000:   0%|          | 0/1403 [00:00<?, ?it/s]

[i] Data/tmdb_api_results_2001.json already exists.


Movies from 2001:   0%|          | 0/728 [00:00<?, ?it/s]

In [48]:
final_2000_df = pd.read_json('Data/tmdb_api_results_2000.json')
final_2000_df.to_csv(f"{FOLDER}final_tmdb_data_2000.csv.gz", compression="gzip", index=False)

In [49]:
final_2001_df = pd.read_json('Data/tmdb_api_results_2001.json')
final_2001_df.to_csv(f"{FOLDER}final_tmdb_data_2001.csv.gz", compression="gzip", index=False)

In [50]:
#import glob
#glob.glob ("Data/*.json")